In [1]:
import kagglehub

data_dir = kagglehub.dataset_download("andrewmvd/dog-and-cat-detection")
print(data_dir)

100%|██████████| 1.03G/1.03G [00:14<00:00, 76.8MB/s]

Extracting files...


/root/.cache/kagglehub/datasets/andrewmvd/dog-and-cat-detection/versions/1


In [2]:
import os
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xml.etree.ElementTree as ET
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tqdm.notebook as tqdm

# Image processing and torchvision
from PIL import Image
from torchvision import transforms, models
from torchvision.models.resnet import ResNet18_Weights, ResNet50_Weights

# Data handling
from torch.utils.data import Dataset, DataLoader

# Scikit-learn utilities
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
class MyDataset(Dataset):
    def __init__(self, annotations_dir, image_dir, transform=None):
        self.annotations_dir = annotations_dir
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = self.filter_images_with_multiple_objects()

    def filter_images_with_multiple_objects(self):
        valid_image_files = []
        for f in os.listdir(self.image_dir):
            if os.path.isfile(os.path.join(self.image_dir, f)):
                img_name = f
                annotation_name = os.path.splitext(img_name)[0] + ".xml"
                annotation_path = os.path.join(self.annotations_dir, annotation_name)
                if self.count_objects_in_annotation(annotation_path) == 1:
                    valid_image_files.append(img_name)
        return valid_image_files

    def count_objects_in_annotation(self, annotation_path):
        try:
            tree = ET.parse(annotation_path)
            root = tree.getroot()
            count = sum(1 for _ in root.findall("object"))
            return count
        except FileNotFoundError:
            return 0

    def parse_annotation(self, annotation_path):
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        label = None
        bbox = None
        image_width = int(root.find("size/width").text)
        image_height = int(root.find("size/height").text)
        for obj in root.findall("object"):
            name = obj.find("name").text
            if label is None:  # Take the first label
                label = name
            # Get bounding box coordinates
            xmin = int(obj.find("bndbox/xmin").text)
            ymin = int(obj.find("bndbox/ymin").text)
            xmax = int(obj.find("bndbox/xmax").text)
            ymax = int(obj.find("bndbox/ymax").text)
            # Normalize bbox coordinates to [0, 1]
            bbox = [
                xmin / image_width,
                ymin / image_height,
                xmax / image_width,
                ymax / image_height,
            ]
        # Convert label to numerical representation (0 for cat, 1 for dog)
        label_num = 0 if label == "cat" else 1 if label == "dog" else -1
        return label_num, torch.tensor(bbox, dtype=torch.float32)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load first image
        img1_file = self.image_files[idx]
        img1_path = os.path.join(self.image_dir, img1_file)
        annotation_name = os.path.splitext(img1_file)[0] + ".xml"
        img1_annotations = self.parse_annotation(
            os.path.join(self.annotations_dir, annotation_name)
        )

        # Load second image (randomly selected)
        idx2 = random.randint(0, len(self.image_files) - 1)
        img2_file = self.image_files[idx2]
        img2_path = os.path.join(self.image_dir, img2_file)
        annotation_name = os.path.splitext(img2_file)[0] + ".xml"
        img2_annotations = self.parse_annotation(
            os.path.join(self.annotations_dir, annotation_name)
        )

        # Open images
        img1 = Image.open(img1_path).convert("RGB")
        img2 = Image.open(img2_path).convert("RGB")

        # Create a horizontally merged image
        merged_w = img1.width + img2.width
        merged_h = max(img1.height, img2.height)
        merged_image = Image.new("RGB", (merged_w, merged_h))
        merged_image.paste(img1, (0, 0))
        merged_image.paste(img2, (img1.width, 0))

        # Adjust bounding boxes for merged image
        merged_annotations = []
        # First image annotations (unchanged)
        merged_annotations.append(
            {"bbox": img1_annotations[1].tolist(), "label": img1_annotations[0]}
        )
        # Second image annotations (adjust coordinates)
        new_bbox = [
            (img2_annotations[1][0] * img2.width + img1.width) / merged_w,
            img2_annotations[1][1] * img2.height / merged_h,
            (img2_annotations[1][2] * img2.width + img1.width) / merged_w,
            img2_annotations[1][3] * img2.height / merged_h,
        ]
        merged_annotations.append({"bbox": new_bbox, "label": img2_annotations[0]})

        # Transform merged image to tensor
        if self.transform:
            merged_image = self.transform(merged_image)
        else:
            merged_image = transforms.ToTensor()(merged_image)

        # Convert annotations to tensors
        annotations = torch.zeros((len(merged_annotations), 5))
        for i, ann in enumerate(merged_annotations):
            annotations[i] = torch.cat(
                (torch.tensor(ann["bbox"]), torch.tensor([ann["label"]]))
            )

        return merged_image, annotations

In [4]:
# Data directory
annotations_dir = os.path.join(data_dir, 'annotations')
image_dir = os.path.join(data_dir, 'images')

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create dataset and dataloaders
dataset = MyDataset(annotations_dir, image_dir, transform=transform)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [5]:
class SimpleYOLO(nn.Module):
    def __init__(self, num_classes):
        super(SimpleYOLO, self).__init__()

        # Backbone using ResNet50
        self.backbone = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        self.num_classes = num_classes

        # Remove the final classification layer of ResNet
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])

        # YOLO head: Linear layer
        # Output size: 2 grid cells, 4 bbox coords + num_classes per cell
        self.fcs = nn.Linear(2048, 2 * 2 * (4 + self.num_classes))

    def forward(self, x):
        # x shape: (batch_size, C, H, W)
        features = self.backbone(x)  # Extract features
        features = F.adaptive_avg_pool2d(features, (1, 1))  # Pool to shape: (batch_size, 2048, 1, 1)
        features = features.view(features.size(0), -1)  # Flatten to shape: (batch_size, 2048)
        features = self.fcs(features)  # Pass through YOLO head
        return features

In [6]:
# Initialize model, criterion, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 2  # Assuming two classes: dog and cat
class_to_idx = {'dog': 0, 'cat': 1}

# Initialize the SimpleYOLO model
model = SimpleYOLO(num_classes=num_classes).to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 134MB/s]


In [7]:
def calculate_loss(output, targets, device, num_classes):
    mse_loss = nn.MSELoss()
    ce_loss = nn.CrossEntropyLoss()
    batch_size = output.shape[0]
    total_loss = 0

    # Reshape output to match grid structure
    output = output.view(batch_size, 2, 2, 4 + num_classes)  # (batch_size, grid_y, grid_x, bbox + class_probs)

    for i in range(batch_size):  # Iterate through batch
        for j in range(len(targets[i])):  # Iterate through objects in an image
            # Determine the grid cell for the object
            bbox_center_x = (targets[i][j][0] + targets[i][j][2]) / 2
            bbox_center_y = (targets[i][j][1] + targets[i][j][3]) / 2
            grid_x = int(bbox_center_x * 2)  # Grid cell X
            grid_y = int(bbox_center_y * 2)  # Grid cell Y

            # Classification Loss
            label_one_hot = torch.zeros(num_classes, device=device)
            label_one_hot[int(targets[i][j][4])] = 1
            classification_loss = ce_loss(output[i, grid_y, grid_x, 4:], label_one_hot)

            # Regression Loss
            bbox_target = targets[i][j][:4].to(device)
            regression_loss = mse_loss(output[i, grid_y, grid_x, :4], bbox_target)

            # No Object Loss
            no_obj_loss = 0
            for other_grid_y in range(2):
                for other_grid_x in range(2):
                    if other_grid_y != grid_y or other_grid_x != grid_x:
                        no_obj_loss += mse_loss(output[i, other_grid_y, other_grid_x, :4],
                                                torch.zeros(4, device=device))

            total_loss += classification_loss + regression_loss + no_obj_loss

    return total_loss / batch_size

In [8]:
def evaluate_model(model, data_loader, device, num_classes):
    all_predictions = []
    all_targets = []
    running_loss = 0.0

    with torch.no_grad():
        for images, targets in tqdm.tqdm(data_loader, desc="Validation", leave=False):
            images = images.to(device)
            output = model(images)
            total_loss = calculate_loss(output, targets, device, num_classes)
            running_loss += total_loss.item()

            # Reshape output to (batch_size, grid_y, grid_x, 4 + num_classes)
            output = output.view(images.shape[0], 2, 2, 4 + num_classes)

            # Collect predictions and targets
            for batch_idx in range(images.shape[0]):
                for target in targets[batch_idx]:
                    bbox_center_x = (target[0] + target[2]) / 2
                    bbox_center_y = (target[1] + target[3]) / 2
                    grid_x = int(bbox_center_x * 2)
                    grid_y = int(bbox_center_y * 2)

                    # Class prediction
                    prediction = output[batch_idx, grid_y, grid_x, 4:].argmax().item()
                    all_predictions.append(prediction)
                    all_targets.append(target[4].item())

    val_loss = running_loss / len(data_loader)
    all_predictions = torch.tensor(all_predictions, device=device)
    all_targets = torch.tensor(all_targets, device=device)
    val_accuracy = (all_predictions == all_targets).float().mean()

    return val_loss, val_accuracy.item()

In [9]:
def train_model(model, train_loader, val_loader, optimizer, num_epochs, device, num_classes):
    best_val_accuracy = 0.0
    train_losses = []
    val_losses = []
    val_accuracies = []

    for epoch in tqdm.tqdm(range(num_epochs), desc="Epochs"):
        model.train()
        running_loss = 0.0

        for images, targets in tqdm.tqdm(train_loader, desc="Batches", leave=False):
            images = images.to(device)
            optimizer.zero_grad()
            output = model(images)
            total_loss = calculate_loss(output, targets, device, num_classes)
            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()

        # Calculate training loss
        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        # Validation phase
        val_loss, val_accuracy = evaluate_model(model, val_loader, device, num_classes)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(
            f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, "
            f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}"
        )

        # Save the best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_model.pth")

    return train_losses, val_losses, val_accuracies

In [11]:
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from PIL import Image
import torch

def inference(model, image_path, transform, device, class_to_idx, threshold=0.5):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    original_width, original_height = image.size

    # Resize the image to match the model's input size (e.g., 448x448)
    resized_image = image.resize((448, 448))
    resized_width, resized_height = resized_image.size

    # Apply transformations
    transformed_image = transform(resized_image).unsqueeze(0).to(device)

    # Forward pass through the model
    with torch.no_grad():
        output = model(transformed_image)

    # Reshape output for a 2x2 grid structure
    output = output.view(1, 2, 2, 4 + len(class_to_idx))

    # Visualization setup
    fig, ax = plt.subplots(1)
    ax.imshow(resized_image)
    ax.axis("off")

    # Process each grid cell
    for grid_y in range(2):
        for grid_x in range(2):
            # Class prediction and bounding box
            class_pred = output[0, grid_y, grid_x, 4:].argmax().item()
            bbox = output[0, grid_y, grid_x, :4].tolist()
            confidence = torch.softmax(output[0, grid_y, grid_x, 4:], dim=0)[class_pred].item()

            # Scale bounding box back to the image dimensions
            x_min = bbox[0] * (resized_width / 2) + grid_x * (resized_width / 2)
            y_min = bbox[1] * (resized_height / 2) + grid_y * (resized_height / 2)
            x_max = bbox[2] * (resized_width / 2) + grid_x * (resized_width / 2)
            y_max = bbox[3] * (resized_height / 2) + grid_y * (resized_height / 2)

            # Draw bounding box and label if confidence exceeds the threshold
            if confidence > threshold:
                rect = patches.Rectangle(
                    (x_min, y_min),
                    x_max - x_min,
                    y_max - y_min,
                    linewidth=1,
                    edgecolor="r",
                    facecolor="none"
                )
                ax.add_patch(rect)
                plt.text(
                    x_min,
                    y_min,
                    f"{list(class_to_idx.keys())[class_pred]}: {confidence:.2f}",
                    fontsize=12,
                    bbox=dict(facecolor="red", alpha=0.5),
                    color="white"
                )

    plt.show()

# Example usage:
# Load the best model
# model.load_state_dict(torch.load("best_model.pth")) # Commented out the original line causing error
num_epochs = 10  # Number of training epochs
train_losses, val_losses, val_accuracies = train_model(model, train_loader, val_loader, optimizer, num_epochs, device, num_classes)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/369 [00:00<?, ?it/s]

Validation:   0%|          | 0/93 [00:00<?, ?it/s]

Epoch 1/10, Train Loss: 1.3672, Validation Loss: 1.2023, Validation Accuracy: 0.8026


Batches:   0%|          | 0/369 [00:00<?, ?it/s]

...

Epoch 9/10, Train Loss: 0.7429, Validation Loss: 0.8467, Validation Accuracy: 0.8704


Batches:   0%|          | 0/369 [00:00<?, ?it/s]

Validation:   0%|          | 0/93 [00:00<?, ?it/s]

Epoch 10/10, Train Loss: 0.7042, Validation Loss: 0.8751, Validation Accuracy: 0.8758


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Study/OD Project/good_1.jpg'

In [12]:
# Inference on a sample image
image_path = "/mnt/c/Study/OD Project/good_1.jpg"
inference(model, image_path, transform, device, class_to_idx, threshold=0.5)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Study/OD Project/good_1.jpg'